<a href="https://colab.research.google.com/github/JemyHo/NLPCW1/blob/Lihang/cw1/F21NL_CW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-17 02:01:04--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8e01beb5480545ac9fe5079369.dl-eu.dropboxusercontent.com/cd/0/inline/CzV1hneTsBiFdbqnhkIksA9yimSq60LMzGdrX8Oi80qs6FOE4CTxpLHkJ74f8AkchJu8tcvwMtyIPBFaAMb-OvoFgT7jAjzu9IMiD31BXHkRKl8CqSUvchilrn_Pd-2xQhHWF8BHC8xyeA5PM5axhgLm/file?dl=1# [following]
--2025-10-17 02:01:05--  https://uc8e01beb5480545ac9fe5079369.dl-eu.dropboxusercontent.com/cd/0/inline/CzV1hneTsBiFdbqnhkIksA9yimSq60LMzGdrX8Oi80qs6FOE4CTxpLHkJ74f8AkchJu8tcvwMtyIPBFaAMb-OvoFgT7jAjzu9IMiD31BXHkRKl8CqSUvchilrn_Pd-2xQhHWF8BHC8xyeA5PM5axhgLm/file?dl=1
Resolving uc8e01beb5480545ac9fe5079369.dl-eu.dropboxusercontent.com (uc8e01beb5480545ac9fe5079369.dl-eu.dro

In [3]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
   creating: wikitext-filtered-full/
  inflating: wikitext-filtered-full/dataset_info.json  
  inflating: wikitext-filtered-full/state.json  
  inflating: wikitext-filtered-full/data-00000-of-00001.arrow  
Archive:  wikitext-filtered-10k.zip
   creating: wikitext-filtered-10k/
  inflating: wikitext-filtered-10k/dataset_info.json  
  inflating: wikitext-filtered-10k/state.json  
  inflating: wikitext-filtered-10k/data-00000-of-00001.arrow  


In [4]:
!pip install datasets
import datasets

In [5]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 74.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 15.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.2
    Uninstalling scipy-1.16.2:
      Successfully uninstalled scipy-1.16.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tsfre

In [1]:
from datasets import load_dataset, Dataset

def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


In [6]:
import nltk, string, os
from gensim.models import Word2Vec
nltk.download('stopwords')
from nltk.corpus import stopwords

STOP = set(stopwords.words('english'))
print(f"{len(STOP)} English stopwords loaded.")

[nltk_data] Downloading package stopwords to /root/nltk_data...


198 English stopwords loaded.


[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
#Step 3

def tokenize_by_space(text):
    """
    Minimal preprocessing (as required in spec):
    - lowercase
    - split by spaces
    - strip punctuation
    - remove stopwords and empty tokens
    """
    tokens = []
    for tok in text.lower().split(" "):
        tok = tok.strip(string.punctuation)
        if not tok or tok in STOP:
            continue
        tokens.append(tok)
    return tokens

def prepare_corpus(raw_ds):
    """Convert HuggingFace Dataset to list of token lists."""
    first = raw_ds[0]
    # find the right text field
    if isinstance(first, dict):
        key = [k for k in first.keys() if isinstance(first[k], str)][0]
        docs = [ex[key] for ex in raw_ds]
    else:
        docs = list(raw_ds)
    return [tokenize_by_space(doc) for doc in docs if doc.strip()]

In [9]:
tokens_small = prepare_corpus(wikitext_small)
# tokens_large = prepare_corpus(wikitext_large)

print("Docs (small):", len(tokens_small))
# print("Docs (large):", len(tokens_large))
print("Example tokens:", tokens_small[0][:20])


Docs (small): 10000
Example tokens: ['senjō', 'valkyria', '3', 'unrecorded', 'chronicles', 'japanese', '戦場のヴァルキュリア3', 'lit', 'valkyria', 'battlefield', '3', 'commonly', 'referred', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', 'tactical', 'role']


In [10]:
SEED = 42
WORKERS = os.cpu_count()

def train_word2vec(sentences, name, vector_size=50, window=5, min_count=5, epochs=5, sg=0):
    print(f"▶ Training {name} | vec={vector_size}, win={window}, min={min_count}, ep={epochs}, sg={sg}")
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,      # 0 = CBOW, 1 = Skip-gram
        workers=WORKERS,
        seed=SEED
    )
    model.train(sentences, total_examples=len(sentences), epochs=epochs)
    print("Vocab size:", len(model.wv))
    model.save(f"w2v_{name}.model")
    print(f"✅ Saved model: w2v_{name}.model\n")
    return model

w2v_small = train_word2vec(tokens_small, "small")
# w2v_large = train_word2vec(tokens_large, "large")


▶ Training small | vec=50, win=5, min=5, ep=5, sg=0


Vocab size: 14554
✅ Saved model: w2v_small.model



In [11]:
import nltk, string, os
from gensim.models import Word2Vec
nltk.download('stopwords')
from nltk.corpus import stopwords

STOP = set(stopwords.words('english'))
print(f"{len(STOP)} English stopwords loaded.")


198 English stopwords loaded.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec


# Load both trained Word2Vec models

w2v_small = Word2Vec.load("w2v_small.model")
# w2v_large = Word2Vec.load("w2v_large.model")
print("Models loaded")
print("small vocab:", len(w2v_small.wv), )#"| large vocab:", len(w2v_large.wv)

Models loaded
small vocab: 14554


In [14]:
# Load WordSim-353 CSV from Google Drive

file_path = '/content/drive/My Drive/wordsim353/combined.csv'

try:
    wordsim_df = pd.read_csv(file_path)
    print("WordSim-353 data loaded successfully.")
    display(wordsim_df.head())
except FileNotFoundError:
    print(f"File not found at {file_path}")
except Exception as e:
    print(f"Error loading file: {e}")

print("Total pairs:", len(wordsim_df))


WordSim-353 data loaded successfully.


,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62


Total pairs: 353


In [15]:
# Helper: compute cosine safely

def compute_cosine(model, w1, w2):
    w1, w2 = w1.lower(), w2.lower()
    try:
        return model.wv.similarity(w1, w2)
    except KeyError:
        return np.nan   # skip missing words

# Compute cosine for the 4 required pairs

pairs = [
    ("plane", "car"),
    ("planet", "sun"),
    ("cup", "article"),
    ("sugar", "approach")
]

print("\n---- wikitext_small ----")
for a, b in pairs:
    print(f"{a:8s}/{b:8s} → {compute_cosine(w2v_small, a, b):.4f}")

# print("\n---- wikitext_large ----")   ####
# for a, b in pairs:   ###
   # print(f"{a:8s}/{b:8s} → {compute_cosine(w2v_large, a, b):.4f}")   #####


# Prepare for full 353-pair evaluation (Step 4)

def evaluate_all(model, df):
    cosines = []
    for _, row in df.iterrows():
        w1, w2 = row['Word 1'], row['Word 2']
        cos = compute_cosine(model, w1, w2)
        cosines.append(cos)
    df[f'{model}'] = cosines
    return cosines

# compute for both models (optional preview)
wordsim_df['cosine_small'] = [
    compute_cosine(w2v_small, w1, w2) for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
]
# wordsim_df['cosine_large'] = [
   # compute_cosine(w2v_large, w1, w2) for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
# ]

display(wordsim_df.head())
print("✅ Cosine columns added – ready for Step 4 (Spearman).")





---- wikitext_small ----
plane   /car      → 0.6534
planet  /sun      → 0.8465
cup     /article  → 0.3436
sugar   /approach → 0.8240


,Word 1,Word 2,Human (mean),cosine_small
0,love,sex,6.77,0.802256
1,tiger,cat,7.35,0.860310
2,tiger,tiger,10.00,1.000000
3,book,paper,7.46,0.614460
4,computer,keyboard,7.62,0.809773


✅ Cosine columns added – ready for Step 4 (Spearman).


In [16]:
# Step 4
from scipy.stats import spearmanr

# Compute Spearman rank-order correlation
rho_small, p_small = spearmanr(wordsim_df['Human (mean)'],
                               wordsim_df['cosine_small'],
                               nan_policy='omit')

#rho_large, p_large = spearmanr(wordsim_df['Human (mean)'],
                             #  wordsim_df['cosine_large'],
                             #  nan_policy='omit')

print(f"Spearman correlation (wikitext_small): {rho_small:.4f} (p={p_small:.4e})")
#print(f"Spearman correlation (wikitext_large): {rho_large:.4f} (p={p_large:.4e})")


Spearman correlation (wikitext_small): 0.1378 (p=2.5968e-02)


In [1]:
#Step 5
import gensim.downloader as api
from google.colab import drive
drive.mount('/content/drive')

# Download model
model = api.load("word2vec-google-news-300")
print("✅ Model loaded!")

# Save it permanently to Drive
model.save("/content/drive/MyDrive/word2vec-google-news-300.model")
print("✅ Model saved to Google Drive!")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[==========----------------------------------------] 21.0% 349.5/1662.8MB downloaded

KeyboardInterrupt: 

In [ ]:
from scipy.stats import spearmanr
import numpy as np

# Safe cosine function for this KeyedVectors model
def compute_cosine_pretrained(model, w1, w2):
    w1, w2 = w1.lower(), w2.lower()
    try:
        return model.similarity(w1, w2)
    except KeyError:
        return np.nan

# Compute cosine for all pairs
wordsim_df['cosine_google'] = [
    compute_cosine_pretrained(google_news_model, w1, w2)
    for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
]

# Calculate Spearman correlation
rho_google, p_google = spearmanr(wordsim_df['Human (mean)'], wordsim_df['cosine_google'], nan_policy='omit')

print(f"Spearman correlation (Google News Word2Vec): {rho_google:.4f} (p={p_google:.4e})")
